In [1]:
import numpy as np
import pandas as pd
import os
import json
from datetime import timedelta
from dateutil import parser
import boto3
import urllib.request
import pickle as pkl
import datetime
import keras
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import regularizers
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from tcn import TCN
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [58]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data"
X = pkl.load(open("x_master.p", "rb"))
Y = pkl.load(open("y_master.p", "rb"))

X.shape

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data


(90487,)

In [40]:
x_bin=[]
y_bin=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(Y):
  if X[i].shape == (160, 4):
      if y in [1,2,3]:
        y_bin.append(1)
        x_bin.append(X[i])
        smoking_event+=1
      elif y >= 20:
        y_bin.append(0)
        x_bin.append(X[i])
        non_smoking_event+=1
        
x_bin=np.array(x_bin)
y_bin=np.array(y_bin)
x_bin.shape

(83369, 160, 4)

In [2]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data"
x_test1 = pkl.load(open("x_test1911.p", "rb"))
y_test1 = pkl.load(open("y_test1911.p", "rb"))

x_test1.shape

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data


(12362,)

In [3]:
x_test=[]
y_test=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(y_test1):
  if x_test1[i].shape == (160, 4):
      if y in [1,2,3]:
        y_test.append(1)
        x_test.append(x_test1[i])
        smoking_event+=1
      elif y >= 20:
        y_test.append(0)
        x_test.append(x_test1[i])
        non_smoking_event+=1
        
x_test=np.array(x_test)
y_test=np.array(y_test)
x_test.shape

(10908, 160, 4)

In [4]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\CNN model\models"
eric_model = tf.keras.models.load_model('model-21-0.9510.hdf5')
pred_prob_eCNN  = eric_model.predict(x_test)
pred_class_eCNN = []
for i in pred_prob_eCNN:
    if i >.05:
        pred_class_eCNN.append(1)
    else:
        pred_class_eCNN.append(0)
print(classification_report(y_test,pred_class_eCNN))
#print(confusion_matrix(y_test,pred_class_eCNN))
#1910 test set

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\CNN model\models
              precision    recall  f1-score   support

           0       1.00      0.88      0.93     10188
           1       0.36      0.96      0.53       720

    accuracy                           0.89     10908
   macro avg       0.68      0.92      0.73     10908
weighted avg       0.96      0.89      0.91     10908



In [5]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\Veena's models"
veena_model = tf.keras.models.load_model('Model_VGG_FReshAir.h5')
pred_prob_vCNN  = veena_model.predict(x_test)
pred_class_vCNN = []
for i in pred_prob_vCNN:
    if i >.05:
        pred_class_vCNN.append(1)
    else:
        pred_class_vCNN.append(0)
print(classification_report(y_test,pred_class_vCNN))
#1910 test set

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\Veena's models
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     10188
           1       0.66      0.64      0.65       720

    accuracy                           0.95     10908
   macro avg       0.82      0.81      0.81     10908
weighted avg       0.95      0.95      0.95     10908



In [6]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\RNN Fresh air"
josh_model = tf.keras.models.load_model('model_rd5_3blocks_-05-0.9830.hdf5')
josh_pred = josh_model.predict(x_test)
pred_class_josh = []
for i in josh_pred:
    if i >.05:
        pred_class_josh.append(1)
    else:
        pred_class_josh.append(0)
print(classification_report(y_test,pred_class_josh))
#1910 test set

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\RNN Fresh air
              precision    recall  f1-score   support

           0       1.00      0.92      0.96     10188
           1       0.46      0.94      0.61       720

    accuracy                           0.92     10908
   macro avg       0.73      0.93      0.78     10908
weighted avg       0.96      0.92      0.93     10908



In [7]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\RNN Fresh air"
josh_model1 = tf.keras.models.load_model('model_rd5_3blocks_lstm100_-02-0.9731.hdf5')
josh_pred1 = josh_model1.predict(x_test)
pred_class_josh1 = []
for i in josh_pred1:
    if i >.05:
        pred_class_josh1.append(1)
    else:
        pred_class_josh1.append(0)
print(classification_report(y_test,pred_class_josh1))
print(confusion_matrix(y_test,pred_class_josh1))
#1910 test set

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\RNN Fresh air
              precision    recall  f1-score   support

           0       1.00      0.83      0.91     10188
           1       0.29      0.98      0.45       720

    accuracy                           0.84     10908
   macro avg       0.65      0.91      0.68     10908
weighted avg       0.95      0.84      0.88     10908

[[8486 1702]
 [  12  708]]


In [8]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\Veena's models"
tcn = tf.keras.models.load_model('Model_TCN_freshAir.h5', custom_objects={'TCN': TCN})
pred_tcn = tcn.predict(x_test)
pred_class_tcn = []
for i in pred_tcn:
    if i >.05:
        pred_class_tcn.append(1)
    else:
        pred_class_tcn.append(0)
print(classification_report(y_test,pred_class_tcn))
#1910 test set

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\Veena's models
              precision    recall  f1-score   support

           0       0.99      0.96      0.97     10188
           1       0.58      0.84      0.68       720

    accuracy                           0.95     10908
   macro avg       0.78      0.90      0.83     10908
weighted avg       0.96      0.95      0.95     10908



In [9]:
predictions = (josh_pred + josh_pred1 + pred_prob_eCNN + pred_tcn +  pred_prob_vCNN)/5

In [10]:
pred_class = []
for i in predictions:
    if i >.05:
        pred_class.append(1)
    else:
        pred_class.append(0)

In [129]:
print(classification_report(y_test,pred_class))
print(confusion_matrix(y_test,pred_class))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93     10188
           1       0.35      0.99      0.52       720

    accuracy                           0.88     10908
   macro avg       0.68      0.93      0.73     10908
weighted avg       0.96      0.88      0.90     10908

[[8882 1306]
 [  10  710]]


In [11]:
pred_hard = np.array(pred_class_josh1).reshape(10908,1) + np.array(pred_class_eCNN).reshape(10908,1) +  np.array(pred_class_tcn).reshape(10908,1)
pred_y_hard = []
for i in pred_hard:
    if i >= 2:
        pred_y_hard.append(1)
    else:
        pred_y_hard.append(0)

In [12]:
print(classification_report(y_test,pred_y_hard))
print(confusion_matrix(y_test, pred_y_hard))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95     10188
           1       0.40      0.97      0.57       720

    accuracy                           0.90     10908
   macro avg       0.70      0.93      0.76     10908
weighted avg       0.96      0.90      0.92     10908

[[9162 1026]
 [  25  695]]


In [47]:
log_reg_data = pd.DataFrame(np.concatenate([y_bin.reshape(83369,1), josh_pred,josh_pred1, pred_prob_eCNN, 
                                           pred_tcn,pred_prob_vCNN], axis = 1),
                            columns = ['y', 'josh1', 'josh2', 'eCNN', 'tcn','vCNN'])
log_reg_data.head()

,y,josh1,josh2,eCNN,tcn,vCNN
0,1.0,0.581738,0.915239,0.997634,0.814264,0.964565
1,1.0,0.943150,0.995341,0.999982,0.873560,0.964565
2,1.0,0.124632,0.754291,0.989980,0.536143,0.964565
3,1.0,0.843354,0.989118,0.999670,0.497567,0.964565
4,1.0,0.788203,0.930470,0.991946,0.348423,1.000000


In [48]:
x_train, x_test, y_train, y_test = train_test_split(log_reg_data.iloc[:,1:6], log_reg_data['y'], test_size=0.1, random_state = 99)
x_train.shape

(75032, 5)

In [49]:
log_reg = LogisticRegression(class_weight ={0: 1, 1: 30}, max_iter = 100).fit(x_train, y_train)
pred_log = log_reg.predict(x_test)
print(classification_report(y_test,pred_log))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8031
         1.0       0.98      0.98      0.98       306

    accuracy                           1.00      8337
   macro avg       0.99      0.99      0.99      8337
weighted avg       1.00      1.00      1.00      8337



In [50]:
nb_mod = GaussianNB().fit(x_train, y_train)
pred_nb = nb_mod.predict(x_test)
print(classification_report(y_test,pred_nb))

              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00      8031
         1.0       0.86      0.98      0.92       306

    accuracy                           0.99      8337
   macro avg       0.93      0.99      0.96      8337
weighted avg       0.99      0.99      0.99      8337



In [51]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data"
x_test1 = pkl.load(open("x_test.p", "rb"))
y_test1 = pkl.load(open("y_test.p", "rb"))

x_test1.shape

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data


(13362,)

In [52]:
x_test=[]
y_test=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(y_test1):
  if x_test1[i].shape == (160, 4):
      if y in [1,2,3]:
        y_test.append(1)
        x_test.append(x_test1[i])
        smoking_event+=1
      elif y >= 20:
        y_test.append(0)
        x_test.append(x_test1[i])
        non_smoking_event+=1
        
x_test=np.array(x_test)
y_test=np.array(y_test)
x_test.shape

(11655, 160, 4)

In [53]:
eCNN_test  = eric_model.predict(x_test)
vCNN_test  = veena_model.predict(x_test)
josh_test = josh_model.predict(x_test)
tcn_test = tcn.predict(x_test)
josh1_test = josh_model1.predict(x_test)

In [54]:
log_reg_test = pd.DataFrame(np.concatenate([y_test.reshape(11655,1),josh_test,josh1_test,eCNN_test, 
                                           tcn_test, vCNN_test], axis = 1),
                            columns = ['y', 'josh1', 'josh2', 'eCNN', 'tcn','vCNN'])
log_reg_test.head()

,y,josh1,josh2,eCNN,tcn,vCNN
0,1.0,0.952438,0.994090,0.988405,9.789606e-01,0.000000e+00
1,1.0,0.961327,0.995621,0.999795,6.332150e-01,9.645648e-01
2,1.0,0.410717,0.942434,0.985313,4.420663e-01,2.086163e-07
3,1.0,0.920080,0.967026,0.999885,3.306146e-01,0.000000e+00
4,1.0,0.000746,0.007206,0.001492,5.960464e-08,0.000000e+00


In [55]:
pred_log = log_reg.predict_proba(np.array(log_reg_test.iloc[:,1:6]))
pred_class_log = []
for i in pred_log[:,1]:
    if i >.5:
        pred_class_log.append(1)
    else:
        pred_class_log.append(0)
print(classification_report(y_test,pred_class_log))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     10591
           1       0.72      0.63      0.67      1064

    accuracy                           0.94     11655
   macro avg       0.84      0.80      0.82     11655
weighted avg       0.94      0.94      0.94     11655



In [56]:
pred_nb_test = nb_mod.predict_proba(np.array(log_reg_test.iloc[:,1:6]))
pred_class_nb = []
for i in pred_nb_test[:,1]:
    if i >.5:
        pred_class_nb.append(1)
    else:
        pred_class_nb.append(0)
print(classification_report(y_test,pred_class_nb))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97     10591
           1       0.71      0.81      0.76      1064

    accuracy                           0.95     11655
   macro avg       0.84      0.89      0.87     11655
weighted avg       0.96      0.95      0.95     11655



In [53]:
pred_nb_test[:,1]

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
       3.03476172e-70, 2.59225025e-70, 2.57744106e-70])

In [48]:
pred_nb_test

array([1., 1., 1., ..., 0., 0., 0.])